# Предсказание победителя матча в Dota 2 по первым 5 минутам

- KDA

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import preproc
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

df_train = pd.read_csv('./features.csv', index_col='match_id')
y = df_train['radiant_win']
# Удалим признаки, связанные с итогами матча (6 штук)
X_feat = df_train.iloc[:, :-6]
X_feat.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [2]:
# Колонки с пропусками
col_miss = X_feat.columns[X_feat.isna().any()].tolist()
miss_val_df = pd.DataFrame({'num_of_nan': len(X_feat)-X_feat[col_miss].count(),
                            'percent_of_nan': (len(X_feat)-X_feat[col_miss].count())/len(X_feat)})
miss_val_df.head()


,num_of_nan,percent_of_nan
first_blood_time,19553,0.201100
first_blood_team,19553,0.201100
first_blood_player1,19553,0.201100
first_blood_player2,43987,0.452402
radiant_bottle_time,15691,0.161380


In [3]:
df_train['first_blood_time'].mean()

97.6880286313838

Новые признаки

In [4]:
# Золото команд к 5 минуте
X_feat['r_team_gold'] = X_feat[['r1_gold', 'r2_gold', 'r3_gold', 'r4_gold', 'r5_gold']].sum(axis=1)
X_feat['d_team_gold'] = X_feat[['d1_gold', 'd2_gold', 'd3_gold', 'd4_gold', 'd5_gold']].sum(axis=1)
# Кол-во убийств к 5 минуте
X_feat['r_team_kills'] = X_feat[['r1_kills', 'r2_kills', 'r3_kills', 'r4_kills', 'r5_kills']].sum(axis=1)
X_feat['d_team_kills'] = X_feat[['d1_kills', 'd2_kills', 'd3_kills', 'd4_kills', 'd5_kills']].sum(axis=1)

#X_feat.head()

In [5]:
(X_feat[['r1_kills', 'r2_kills', 'r3_kills', 'r4_kills', 'r5_kills']].sum(axis=1)/ \
X_feat[['r1_deaths', 'r2_deaths', 'r3_deaths', 'r4_deaths', 'r5_deaths']].sum(axis=1)) \
    .value_counts().sort_index(ascending=False).max()

16943

In [6]:
# Заменяем пропуски на нули
X_feat.fillna(0, inplace=True)
# Список категориальных признаков
categ_feat = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero',  'r4_hero', 'r5_hero',
              'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
# Количество персонажей
num_heroes = preproc.num_heroes(X_feat)
# добавим num_heroes признаков (если "0" - персонаж не участвовал в игре, "1"- Radiant, "-1" - Dire)
X_feat = preproc.heroes_bag(X_feat, num_heroes)
X_feat.drop(categ_feat[1:], axis=1, inplace=True)
# Закодируем lobby_type
X_feat = preproc.ohe(X_feat, ['lobby_type'], True)

#X_feat.head()

### Возьмем в качестве baseline логистическую регрессию (как в финальном задании курса)

In [7]:
cv = KFold(n_splits=5, shuffle=True)
clf = LogisticRegression
# Нормализация
scaler = StandardScaler()
X_feat = pd.DataFrame(data=scaler.fit_transform(X_feat), 
                    index=X_feat.index, columns=X_feat.columns)

In [11]:
C_regul = 0.1
clf = LogisticRegression(penalty='l2', C = C_regul).fit(X_feat, y)
cross_val_score(clf, X_feat, y, cv=cv, scoring='roc_auc').mean()

XGboost

In [15]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(X_feat, y, test_size=0.3)

xgb_model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False,
                            n_estimators=200,
                            learning_rate=0.1, 
                            reg_lambda=0.1)

xgb_model.fit(X_train, y_train, 
                early_stopping_rounds=5, 
                eval_set=[(X_val, y_val)], 
                verbose=False)
roc_auc_score(y_val, xgb_model.predict_proba(X_val)[:, 1])

[16:35:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7420022567309943